In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from source.datasets.fast_datasets import *
from source.utils.load_utils import split_and_load
#import torch

genre_dataset = GTZANFastDataset()

# deezer_dataset = DeezerFastDataset()

train_dataloader, val_dataloader = split_and_load(genre_dataset, workers=4, batch_size=4, split_size=0.75)

In [3]:
from source.models_task_specific.mb_classification import MusicBertClassifier
from source.models_task_specific.vggish_classification import VGGishClassifier

num_classes = 10
evals = 0.04

model = MusicBertClassifier(num_classes, RNN=True, num_encoder_layers=4).cuda()
# model = VGGishClassifier(num_classes, pretrained=False).cuda()
#model.BERT.load_pretrained()
#model.load_model("models/music_bert_audioset_4layers.pth")

In [10]:
model.train_model(train_dataloader, val_dataloader, epochs = 100, eval_per_epoch=evals)

RuntimeError: DataLoader worker (pid(s) 36350, 36355, 36356) exited unexpectedly

In [ ]:
from source.utils.generic_utils import allDone

allDone()

In [9]:
%matplotlib widget
import numpy as np

from source.utils.plot_utils import plot_curve

def smooth(y, box_pts):
    box = np.ones(box_pts)/box_pts
    y_smooth = np.convolve(y, box, mode='same')
    return y_smooth[box_pts:-box_pts]

loss = smooth(model.loss_curve.cpu().numpy(), 20)

plot_curve(loss, 1, color="red")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
%matplotlib widget

# plot_curve(saved_curve, 1, color="red")
# plot_curve(saved_curve_2, 1, color="blue")
# plot_curve(saved_curve_3, 1, color="green")
# plot_curve(saved_curve_4, 1, color="yellow")
plot_curve(model.validation_curve.cpu().numpy(), 1, color="purple")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
import numpy as np

np.mean(model.validation_curve.cpu().numpy()[-4:])

0.82459676